In [131]:
import pandas as pd
import plotly.express as px
import folium
df = pd.read_csv('/home/thiagoferes/projects/CDS/FTC/dataset/train.csv')
df1 = df.copy()


# convertendo Delivery_person_Age para int
linhas_selecionadas = df1['Delivery_person_Age'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)

# convertendo Delivery_person_Ratings para float
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype(float)

# convertendo Order_Date para datetime
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format=('%d-%m-%Y'))

# convertendo multiple_deliveries para int
linhas_selecionadas = df1['multiple_deliveries'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype(int)

# removendo espacos dentro de strings
df1.loc[:, 'ID'] = df1.loc[:, 'ID'].str.strip()
df1.loc[:, 'Road_traffic_density'] = df1.loc[:, 'Road_traffic_density'].str.strip()
df1.loc[:, 'Type_of_order'] = df1.loc[:, 'Type_of_order'].str.strip()
df1.loc[:, 'Type_of_vehicle'] = df1.loc[:, 'Type_of_vehicle'].str.strip()
df1.loc[:, 'City'] = df1.loc[:, 'City'].str.strip()

#removendo linhas 'NaN' do Road_traffic_density
#df1 = df1.loc[df1['Road_traffic_density'] != 'NaN', :]
#linhas_selecionadas = (df1['Road_traffic_density'] != 'NaN ')
#df1 = df1.loc[linhas_selecionadas, :].copy()

#removendo linhas 'NaN' do City
#df1 = df1.loc[df1['City'] != 'NaN', :]
#linhas_selecionadas = (df1['City'] != 'NaN ')
#df1 = df1.loc[linhas_selecionadas, :].copy()

#limpando a coluna de time taken
df1['Time_taken(min)'] = df1['Time_taken(min)'].apply( lambda x: x.split('(min) ')[1])
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)

In [121]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30


# Exercicios

## Do lado da empresa:


### 1. Quantidade de pedidos por dia.


In [78]:
cols = ['ID', 'Order_Date']
df_aux = df1.loc[:, cols].groupby(['Order_Date']).count().reset_index()

px.bar(df_aux, x='Order_Date', y='ID')



### 2. Quantidade de pedidos por semana.


In [79]:
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U')

df_aux = df1.loc[:, ['week_of_year', 'Order_Date']].groupby(['week_of_year']).count().reset_index()
px.line(df_aux, x='week_of_year', y='Order_Date')


### 3. Distribuição dos pedidos por tipo de tráfego.


In [80]:
df_aux = df1.loc[:, ['ID', 'Road_traffic_density']].groupby(['Road_traffic_density']).count().reset_index()
df_aux['delivery_perc'] = df_aux['ID']/df_aux['ID'].sum()
px.pie(df_aux, values='delivery_perc', names='Road_traffic_density')

### 4. Comparação do volume de pedidos por cidade e tipo de tráfego.


In [81]:
df_aux = df1.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()
px.scatter(df_aux, x='City', y='Road_traffic_density', size='ID', color='City')

### 5. A quantidade de pedidos por entregador por semana.

In [82]:
df_aux1 = df1.loc[:, ['week_of_year', 'ID']].groupby(['week_of_year']).count().reset_index()
df_aux2 = df1.loc[:, ['week_of_year', 'Delivery_person_ID']].groupby(['week_of_year']).nunique().reset_index()

df_aux = pd.merge( df_aux1, df_aux2, how='inner')
df_aux['order_by_deliver'] = df_aux['ID'] / df_aux['Delivery_person_ID']
px.line(df_aux, x='week_of_year', y='order_by_deliver')

### 6. A localização central de cada cidade por tipo de tráfego.

In [83]:
df_aux = df1.loc[:, ['City', 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']].groupby(['City', 'Road_traffic_density']).median().reset_index()

map = folium.Map()

for index, location_info in df_aux.iterrows():
    folium.Marker([location_info['Delivery_location_latitude'], 
                   location_info['Delivery_location_longitude']], 
                   popup=location_info[['City', 'Road_traffic_density']]).add_to(map)

map

## Do lado do entregador:

### 1. A menor e maior idade dos entregadores.


In [84]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24,11
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33,12
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26,11
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21,14
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30,12


In [132]:
print(f'A maior idade eh:', df1.loc[:, 'Delivery_person_Age'].max())
print(f'A menor idade eh:', df1.loc[:, 'Delivery_person_Age'].min())

A maior idade eh: 50
A menor idade eh: 15


### 2. A pior e a melhor condição de veículos.


In [86]:
print(f'A melhor condicao do veiculo:', df1.loc[:, 'Vehicle_condition'].max())
print(f'A pior condicao do veiculo:', df1.loc[:, 'Vehicle_condition'].min())

A melhor condicao do veiculo: 2
A pior condicao do veiculo: 0


### 3. A avaliação média por entregador.


In [87]:
df1.loc[:, ['Delivery_person_Ratings', 'Delivery_person_ID']].groupby(['Delivery_person_ID']).mean().reset_index()

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000
...,...,...
1315,VADRES19DEL02,4.630357
1316,VADRES19DEL03,4.670270
1317,VADRES20DEL01,4.620370
1318,VADRES20DEL02,4.591111


### 4. A avaliação média e o desvio padrão por tipo de tráfego.


In [106]:
df_avg_std_rating_by_traffic = df1.loc[:, ['Delivery_person_Ratings', 'Road_traffic_density']].groupby(['Road_traffic_density']).agg({'Delivery_person_Ratings': ['mean', 'std']})
df_avg_std_rating_by_traffic.columns = ['delivery_mean', 'delivery_std']
df_avg_std_rating_by_traffic.reset_index()

,Road_traffic_density,delivery_mean,delivery_std
0,High,4.652538,0.272699
1,Jam,4.594428,0.329531
2,Low,4.645947,0.337211
3,Medium,4.660335,0.273956


### 5. A avaliação média e o desvio padrão por condições climáticas.


In [107]:
df_avg_std_rating_by_weather = df1.loc[:, ['Delivery_person_Ratings', 'Weatherconditions']].groupby(['Weatherconditions']).agg({'Delivery_person_Ratings': ['mean', 'std']})
df_avg_std_rating_by_weather.columns = ['delivery_mean', 'delivery_std']
df_avg_std_rating_by_weather.reset_index()

,Weatherconditions,delivery_mean,delivery_std
0,conditions Cloudy,4.651945,0.281016
1,conditions Fog,4.653087,0.274974
2,conditions Sandstorms,4.612144,0.310527
3,conditions Stormy,4.612569,0.312621
4,conditions Sunny,4.656164,0.395057
5,conditions Windy,4.616844,0.304087


### 6. Os 10 entregadores mais rápidos por cidade.


In [149]:
df2 = df1.loc[:, ['Delivery_person_ID', 'Time_taken(min)', 'City']].groupby(['City', 'Delivery_person_ID']).mean().sort_values(['Time_taken(min)', 'City']).reset_index()

df_aux1 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_aux2 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_aux3 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df3 = pd.concat([df_aux1, df_aux2, df_aux3]).reset_index(drop=True)

df3

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,KNPRES03DEL02,15.750000
1,Metropolitian,ALHRES02DEL02,17.800000
2,Metropolitian,KNPRES01DEL01,19.125000
3,Metropolitian,KOLRES01DEL03,19.125000
4,Metropolitian,KOCRES02DEL02,19.250000
5,Metropolitian,KOCRES08DEL03,19.800000
6,Metropolitian,KOCRES16DEL03,20.000000
7,Metropolitian,KOCRES02DEL03,20.375000
8,Metropolitian,GOARES14DEL01,20.400000
9,Metropolitian,DEHRES20DEL03,20.428571


### 7. Os 10 entregadores mais lentos por cidade.

In [148]:
df2 = df1.loc[:, ['Delivery_person_ID', 'Time_taken(min)', 'City']].groupby(['City', 'Delivery_person_ID']).mean().sort_values(['Time_taken(min)', 'City'], ascending=False).reset_index()

df_aux1 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_aux2 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_aux3 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df3 = pd.concat([df_aux1, df_aux2, df_aux3]).reset_index(drop=True)

df3

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AGRRES02DEL01,39.428571
1,Metropolitian,AURGRES11DEL03,38.500000
2,Metropolitian,KOLRES03DEL03,38.142857
3,Metropolitian,LUDHRES17DEL03,37.000000
4,Metropolitian,ALHRES18DEL02,36.666667
5,Metropolitian,GOARES08DEL03,36.000000
6,Metropolitian,ALHRES12DEL01,35.000000
7,Metropolitian,KNPRES06DEL01,34.555556
8,Metropolitian,ALHRES010DEL01,34.500000
9,Metropolitian,KOCRES09DEL02,34.333333


In [145]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30
